In [ ]:
# !pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import *
from keras.models import *
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.layers import Layer
import gensim
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # Lowercase Text
    data_frame['Text'] = data_frame['Text'].str.lower()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

# Data

### Twitter 1

In [ ]:
# ## Preparing the data
# ## Twitter 10000
Twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/Twitter/twitter-suicidal_data_10000.csv"
df = pd.read_csv(Twitter_path, encoding='latin-1')
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})
df = preprocessing(df)
df


### Twitter 2

In [ ]:
twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/suicidal-tendency-tweets.csv"
df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})

df = preprocessing(df)
df

In [ ]:
print(list(df['Label']).count(1), list(df['Label']).count(0))

## Reddit SNS

In [ ]:
# ## Preparing the data

Reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_non suicide  suicide/Suicide_Detection.csv"

## reading the abovementioned datasets
# df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'])
df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'], nrows = 20000)
df_r = df_r.rename(columns={'text': 'Text', 'class': 'Label'})

df = preprocessing(df_r)

label_dict = {'suicide': 1, 'non-suicide': 0}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df['Label']

df

In [ ]:
print(list(df['Label']).count(1), list(df['Label']).count(0))

## Reddit _ SD

In [ ]:
## Preparing the data
reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_depressionSuicide/reddit_depression_suicidewatch.csv" 

df = pd.read_csv(reddit_path, encoding='latin-1')
df = df.rename(columns={'text':'Text', 'label':'Label'}) 
label_dict = {'depression': 0, 'SuicideWatch': 1}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df = preprocessing(df)
df

Text  Label  \
0      i recently went through a breakup and she said...      0   
1      i do not know how to navigate these feelings  ...      0   
2      so i have been with my bf for 5 months   and h...      0   
3      i am so exhausted of this  just when i think i...      1   
4      i have been severly bullied since i was 5 till...      0   
...                                                  ...    ...   
20358  i took 50mg of seroquel a few hours after i dr...      1   
20359  that is what has happened to me last week  and...      0   
20360  ever just feel alone in a house full of people...      0   
20361  politicians  neighbors  corporations  society ...      0   
20362  i feel like i am just existing  but for what  ...      0   

                                                  Tokens  Length  
0      [i, recently, went, through, a, breakup, and, ...     121  
1      [i, do, not, know, how, to, navigate, these, f...     308  
2      [so, i, have, been, with, my, bf, for, 5, mont...     131  
3      [i, am, so, exhausted, of, this, just, when, i...     349  
4      [i, have, been, severly, bullied, since, i, wa...     198  
...                                                  ...     ...  
20358  [i, took, 50mg, of, seroquel, a, few, hours, a...      35  
20359  [that, is, what, has, happened, to, me, last, ...     119  
20360  [ever, just, feel, alone, in, a, house, full, ...      20  
20361  [politicians, neighbors, corporations, society...      31  
20362  [i, feel, like, i, am, just, existing, but, fo...     133  

[20363 rows x 4 columns]

# Word Embeddings

## Word2Vec model

In [ ]:
# word2vec embedding
wd2vc_model = gensim.models.Word2Vec(df['Tokens'], min_count = 1, size = 300, window = 5)
wd2vc_model.train(df['Tokens'],epochs=10,total_examples=len(df['Tokens']))

In [ ]:
# words most similar to a given word.
similar_words = wd2vc_model.wv.most_similar('kill')
print(similar_words)

[('killing', 0.7109107971191406), ('killed', 0.7084794044494629), ('isolate', 0.6956068873405457), ('offing', 0.6608023643493652), ('shoot', 0.659714937210083), ('afirm', 0.6361360549926758), ('anythingfuckmaybe', 0.6252297163009644), ('prostrate', 0.6112207174301147), ('convince', 0.6095135807991028), ('die', 0.6069653034210205)]


In [ ]:
## Creating the embedding matrix

vocab = wd2vc_model.wv.vocab
print("The total number of words are : ", len(vocab))
vocab = list(vocab.keys())

word_vec_dict = {}
for word in vocab:
    word_vec_dict[word] = wd2vc_model.wv.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))  # should come equal to vocab size

max_length = max(df['Length'])
print("maximum length = ", max_length)

tok = Tokenizer()
tok.fit_on_texts(df['Tokens'])
vocab_size = len(tok.word_index) + 1

encd_rev = tok.texts_to_sequences(df['Tokens'])

embed_dim = 300
pad_rev = pad_sequences(encd_rev, maxlen=max_length, padding='post')
print("pad_shape = ", pad_rev.shape)

# the embedding matrix
embed_matrix = np.zeros(shape=(vocab_size, embed_dim))
print(embed_matrix.shape)
for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None: 
        embed_matrix[i] = embed_vector

In [ ]:
## Spliting the data
X, X_test, y, y_test = train_test_split(pad_rev, df['Label'], shuffle=True, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)
print("Validation = ",X_val.shape, y_val.shape)


# Implemented model

### Reshaping the vectors

In [ ]:
X_train_reshaped = X_train.reshape(-1, 1, X_train.shape[1])
X_test_reshaped  = X_test.reshape(-1, 1, X_train.shape[1])
X_val_reshaped = X_val.reshape(-1, 1, X_train.shape[1])

In [ ]:
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)
y_val_reshaped = y_val.values.reshape(-1, 1)

In [ ]:
print("Train = ", X_train_reshaped.shape, y_train_reshaped.shape)
print("Test = ", X_test_reshaped.shape, y_test_reshaped.shape)
print("Validation = ", X_val_reshaped.shape, y_val_reshaped.shape)

In [ ]:
n_inputs = X_train_reshaped.shape[2]
n_inputs

## Al-BTCN model

In [ ]:
# a custome Attention layer
class Attention(Layer):
  def __init__ (self, return_sequences=True):
    self.return_sequences = return_sequences
    super(Attention, self).__init__()

  def build (self, input_shape):
    self.W = self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
    self.b = self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
    super(Attention, self).build(input_shape)

  def call(self, x):
    e = K.tanh(K.dot(x, self.W)+self.b)
    a = K.softmax(e, axis=1)
    output = x*a
    if self.return_sequences:
      return output
    return K.sum(output, axis = 1)


In [ ]:
# !pip install keras==2.0.8

In [ ]:
# !pip install keras-tcn
!pip install keras-tcn --no-dependencies  # without the dependencies if you already have TF/Numpy.

In [ ]:
from tcn import TCN, tcn_full_summary
from keras.layers import Bidirectional
from tensorflow.keras.initializers import RandomNormal, Constant

def tcn_model(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  weights = [emb_matrix],
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    x = LSTM(100, activation='tanh', stateful=False, return_sequences = True)(x)
    x = Dense(200, activation="relu")(x)
    x = Bidirectional(TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1'))(x)
    x = Bidirectional(TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2'))(x)
    x = Attention(return_sequences=True)(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_tcn = tcn_model(kernel_size = 8, activation='relu', input_dim = vocab_size, output_dim=300, max_length = max_length, emb_matrix = embed_matrix)

In [ ]:
epochs = 30
batch_size = 64
r = model_tcn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
# Plot the AL-BTCN model
from tensorflow.keras.utils import plot_model
plot_model(model_tcn, 'AL-BTCN.png', show_shapes=True)

## Prediction

In [ ]:
#Prediction on the test data
pred = model_tcn.predict(X_test)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
  if pred[item] >= 0.5:
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    y_pred[item] = 1
  else:
    y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)